In [86]:
import numpy as np
import os
import glob
import SimpleITK as sitk
from skimage import measure
import nibabel as nib
import itk
from itkwidgets import view
import seaborn as sns
from mitok.utils.mdicom import SERIES

def connected_domain(image, mask=True):
    cca = sitk.ConnectedComponentImageFilter()
    cca.SetFullyConnected(True)
    _input = sitk.GetImageFromArray(image.astype(np.uint8))
    output_ex = cca.Execute(_input)
    stats = sitk.LabelShapeStatisticsImageFilter()
    stats.Execute(output_ex)
    num_label = cca.GetObjectCount()
    num_list = [i for i in range(1, num_label+1)]
    area_list = []
    for l in range(1, num_label +1):
        area_list.append(stats.GetNumberOfPixels(l))
    num_list_sorted = sorted(num_list, key=lambda x: area_list[x-1])[::-1]
    largest_area = area_list[num_list_sorted[0] - 1]
    final_label_list = [num_list_sorted[0]]

    for idx, i in enumerate(num_list_sorted[1:]):
        if area_list[i-1] >= (largest_area//10):
            final_label_list.append(i)
        else:
            break
    output = sitk.GetArrayFromImage(output_ex)

    for one_label in num_list:
        if  one_label in final_label_list:
            continue
        x, y, z, w, h, d = stats.GetBoundingBox(one_label)
        one_mask = (output[z: z + d, y: y + h, x: x + w] != one_label)
        output[z: z + d, y: y + h, x: x + w] *= one_mask

    if mask:
        output = (output > 0).astype(np.uint8)
    else:
        output = ((output > 0)*255.).astype(np.uint8)
    return output

def connected_component(image):
    # 标记输入的3D图像
    label, num = measure.label(image, connectivity=2, return_num=True)
    if num < 1:
        return [], image
        
    # 获取对应的region对象
    region = measure.regionprops(label)
    # 获取每一块区域面积并排序
    num_list = [i for i in range(1, num+1)]
    area_list = [region[i-1].area for i in num_list]
    num_list_sorted = sorted(num_list, key=lambda x: area_list[x-1])[::-1]
    # 去除面积较小的连通域
    '''
    if len(num_list_sorted) > 3:
        # for i in range(3, len(num_list_sorted)):
        for i in num_list_sorted[3:]:
            # label[label==i] = 0
            label[region[i-1].slice][region[i-1].image] = 0
        num_list_sorted = num_list_sorted[:3]
    '''
    return num_list_sorted, label
   
def np_count(nparray, x):
    i = 0
    for n in nparray:
        if n == x:
            i += 1
    return i

def get_volume(vessel_num, vessel_data):
    vessel_volume = []
    for i in range(len(vessel_num)):
        vessel_volume.append(np_count(vessel_data[np.nonzero(vessel_data)], vessel_num[i]))
    return vessel_volume

def remove_mix_region(num, volume, out_data):
    after_num = []
    after_volume = []
    for i in range(len(num)):
        if volume[i] > 0:
            after_num.append(num[i])
            after_volume.append(volume[i])
        else:
            out_data[out_data==num[i]] = 0
    return after_num, after_volume, out_data

def save_nii(plaque, data):
    dealt_plaque_dir = plaque.split('mask_source/')[0]
    data[np.nonzero(data)] = 1
    plaque_nii = nib.Nifti1Image(data, np.eye(4))
    nib.save(plaque_nii, os.path.join(dealt_plaque_dir, 'mask_source/mask_plaque_dealt.nii.gz'))            
    
def get_brightness(after_num, after_data, img_tensor):
    brightness = []
    for i in after_num:
        brightness.append(round(img_tensor[after_data==i].mean()))
        #print(img_tensor[after_data==i])
    return brightness

#print(plaques)
#plaques_dir = os.path.join(plaques.split('mask_source/')[0], 'mask_source/mask_plaque.nii.gz')
#data = sitk.ReadImage('/mnt/DrwiseDataNFS/drwise_runtime_env/data1/inputdata/1036623/FCE495EE/784DB1F2_CTA/mask_source/mask_plaque_dealt.nii.gz')
#data = sitk.GetArrayFromImage(data)
#print(data.shape)
#img = itk.imread('/mnt/DrwiseDataNFS/drwise_runtime_env/data1/inputdata/1022837/5A047805/62D38185_CTA/mask_source/mask_plaque.nii.gz')#/mnt/DrwiseDataNFS/drwise_runtime_env/data1/inputdata/1022837/5A047805/62D38185_CTA/mask_source
#view(img)

def show_data(path):
    print(path)
    img = sitk.ReadImage(path)
    data = sitk.GetArrayFromImage(img)
    print(data.shape)
    data = np.unique(data)
    print(data)

#broken_vessels = [1073332/5F654861, 1073332/5F653FE9, 1036627/CBA29632, 1036604/65669469, 1073308/CE363116(F), 1036623/784D418E, 1073309/AB512F17, 1036609/E36BF3A6, 
#1073297/B1874945, 1073318/AF7B8171, 1073318/AF7B89E9, 1036617/6337C565, 1036617/6337BD02, 1073300/2B82D8BC, 1073298/CF547628, 1022836/56847483]
broken_vessel = '1036627/CBA29632'
#plaque_name = '/1036603/88130C9F/D6F78C10_CTA' #'/mnt/DrwiseDataNFS/drwise_runtime_env/data1/inputdata/'+plaque_name+'/mask_source/mask_vessel_dealt.nii.gz'
vessel_name = glob.glob(os.path.join('/mnt/DrwiseDataNFS/drwise_runtime_env/data1/inputdata/', broken_vessel.split('/')[0], '*', broken_vessel.split('/')[1]+'_CTA', 'mask_source/mask_vessel.nii.gz'))
vessel_nii = nib.load(vessel_name[0])
plaque_nii = nib.load('/mnt/users/ffr_plaque_mask/'+broken_vessel+'/mask_plaque.nii.gz')
vessel_noplaque_repaired_nii = nib.load('/mnt/users/ffr_plaque_mask/'+broken_vessel+'/mask_vessel_dealt.nii.gz')
vessel_plaque_repaired_nii = nib.load('/mnt/users/ffr_plaque_mask/'+broken_vessel+'/mask_repaired_vessel_plaque.nii.gz')
vessel_plaque_pipeline_nii = nib.load('/mnt/users/ffr_plaque_mask/'+broken_vessel+'/mask_pipeline_vessel_plaque_middlepoint.nii.gz')
vessel_plaque_pipeline_view_nii = nib.load('/mnt/users/ffr_plaque_mask/'+broken_vessel+'/mask_pipeline_vessel_plaque.nii.gz')
#show_data('/mnt/DrwiseDataNFS/drwise_runtime_env/data1/inputdata/1073332/B5664E68/5F654861_CTA/mask_source/mask_vessel.nii.gz')
show_data('/mnt/users/ffr_plaque_mask/1073332/5F654861/mask_plaque.nii.gz')
vessel = vessel_nii.get_fdata()
plaque = plaque_nii.get_fdata()
vessel_noplaque_repaired = vessel_noplaque_repaired_nii.get_fdata()
vessel_plaque_repaired = vessel_plaque_repaired_nii.get_fdata()
vessel_plaque_pipeline = vessel_plaque_pipeline_nii.get_fdata()
vessel_plaque_pipeline_view = vessel_plaque_pipeline_view_nii.get_fdata()
data = sitk.ReadImage('/mnt/users/ffr_plaque_mask/'+broken_vessel+'/mask_vessel_dealt.nii.gz')
data = sitk.GetArrayFromImage(data)
data[data==2] = 1
data_num, out_data = connected_component(data)
print('vessel_data', data_num, get_volume(data_num, out_data))
pts0 = np.stack(np.where(vessel>0), axis=1)
pts1 = np.stack(np.where(plaque>0), axis=1)
pts2 = np.stack(np.where(vessel_noplaque_repaired==1), axis=1)
pts3 = np.stack(np.where(vessel_noplaque_repaired==2), axis=1)
pts4 = np.stack(np.where(vessel_plaque_repaired>0), axis=1)
pts5 = np.stack(np.where(vessel_plaque_repaired==1), axis=1)
pts6 = np.stack(np.where(vessel_plaque_repaired==2), axis=1)
pts7 = np.stack(np.where(vessel_plaque_pipeline_view==1), axis=1)
pts8 = np.stack(np.where(vessel_plaque_pipeline_view==3), axis=1)
pts9 = np.stack(np.where(vessel_plaque_pipeline==1), axis=1)
pts10 = np.stack(np.where(vessel_plaque_pipeline==2), axis=1)
pts11 = np.stack(np.where(vessel_plaque_pipeline==3), axis=1)
#view(point_sets=[pts1, pts2, pts3])
view(point_sets=[pts7, pts8])
#view(point_sets=[pts9, pts10, pts11])

/mnt/users/ffr_plaque_mask/1073332/5F654861/mask_plaque.nii.gz
(275, 512, 512)
[0 1]
vessel_data [1, 6, 5, 2, 9, 3, 7, 8, 10, 4] [1128077, 2328, 491, 15, 10, 9, 5, 2, 1, 1]


Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157 , 0.        , 0.        ],
   …

In [87]:
#view(point_sets=[pts1])

In [88]:
view(point_sets=[pts9, pts10, pts11])

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157 , 0.        , 0.        ],
   …

In [70]:
#vessel_nii = nib.load('/mnt/DrwiseDataNFS/drwise_runtime_env/data1/inputdata/'+'1022832/FA7B2672/E561C900_CTA/'+'mask_source/mask_vessel.nii.gz')
#plaque_nii = nib.load('/mnt/users/ffr_plaque_mask/'+'1022832/E561C900'+'/mask_plaque.nii.gz')
#vessel = vessel_nii.get_fdata()
#plaque = plaque_nii.get_fdata()
#pts0 = np.stack(np.where(vessel>0), axis=1)
#pts1 = np.stack(np.where(plaque>0), axis=1)
#view(point_sets=[pts0, pts1])